In [9]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
import time

RI_1 = [[196,198,200,206,209,211], [145,150,704,706,707,711,758,767,768]]

# what are the ranges of the axes
x_ax = 640
y_ax = 350

# what are the thresholds
radius_thr = 50
angle_thr = 11.25

# GET EVERYTHING FROM CSV INTO LISTS
def csv_into_lists(csv_file):
    with open(csv_file, "r") as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        track = header.index("track")
        frame = header.index("frame_idx")
        snout_x = header.index("snout.x")
        snout_y = header.index("snout.y")
        head_x = header.index("head.x")
        head_y = header.index("head.y")
        body_x = header.index("body_center.x")
        body_y = header.index("body_center.y")
        tail_start_x = header.index("tail_start.x")
        tail_start_y = header.index("tail_start.y")

        resident, intruder = [], []
        for row in csv_reader:
            if row[track] == "Resident":
                resident.append([row[frame], row[snout_x], row[snout_y], row[head_x], row[head_y], 
                                 row[body_x], row[body_y], row[tail_start_x], row[tail_start_y]])
            elif row[track] == "Intruder":
                intruder.append([row[frame], row[snout_x], row[snout_y], row[head_x], row[head_y], 
                                 row[body_x], row[body_y], row[tail_start_x], row[tail_start_y]])
    return resident, intruder

In [10]:
# CLEAN UP THE CSV LISTS
# frames with no instaces are filled with -1
def fill_empty_frames(incomplete, max_frame):
    collected_frames = [frame[0] for frame in incomplete]
    missing_frames = [frame for frame in range(max_frame) if frame not in collected_frames]
    missing_frames = [[frame,-1,-1,-1,-1,-1,-1,-1,-1] for frame in missing_frames]
    unsorted_frames = incomplete + missing_frames
    sorted_frames = sorted(unsorted_frames, key=lambda x: x[0])
    return sorted_frames

# put x and y coordinates of points into nice schemata [snout, head, body, tail] and reverse y coordinates
def y_rev(y_coo):
    return y_ax - y_coo if y_coo != -1 else -1

def clean_up_lists(resident, intruder):  
    # turn everything into float numbers
    resi = [[float(value) if value != "" else -1 for value in frame] for frame in resident]
    intr = [[float(value) if value != "" else -1 for value in frame] for frame in intruder]
    
    # what is the maximum number of frames (in resi or intr)
    max_frame = int(max([max([frame[0] for frame in resi]), max([frame[0] for frame in intr])]))
    
    resi = fill_empty_frames(resi, max_frame)
    intr = fill_empty_frames(intr, max_frame)
    
    # check amount of frames per animal and delete last one if different
    if len(resi) > len(intr):
        del resi[-1]
    elif len(resi) < len(intr):
        del(intr[-1])

    # delete frame[0], not necessary after we filled empty frames
    resi = [frame[1:] for frame in resi]
    intr = [frame[1:] for frame in intr]

    # make everything into nice order
    resi_coo = np.array([[[frame[0], y_rev(frame[1])], [frame[2], y_rev(frame[3])], [frame[4], y_rev(frame[5])], 
                          [frame[6], y_rev(frame[7])]] for frame in resi])
    intr_coo = np.array([[[frame[0], y_rev(frame[1])], [frame[2], y_rev(frame[3])], [frame[4], y_rev(frame[5])], 
                          [frame[6], y_rev(frame[7])]] for frame in intr])
    return resi_coo, intr_coo

In [11]:
# ANIMALS IN RADIUS?
# create list with 0 (not in radius) or 1 (in radius) for each frame, if instance is missing it`s in radius
def points_in_radius(resi, intr):
    distance = np.sqrt((intr[0] - resi[0])**2 + (intr[1] - resi[1])**2)
    return True if distance <= radius_thr else False

def animals_in_proximity(f, frame, intr_coo):
    # return True if animals are in radius, False if not
    # check if (either in resi or intr), all body parts are filled with -1
    if np.all(frame == -1) or np.all(intr_coo[f] == -1):
        return True
    
    # find a resi body part with no -1 in it, go from snout to tail
    resi_pos = np.array(frame[np.where(frame != -1)][0:2])

    # check for every intr body part if its distance to resi_pos is in radius
    for intr_part in intr_coo[f]:
        if not np.isin(-1, intr_part) and points_in_radius(resi_pos, intr_part):
            return True
    return False

def animals_in_radius(resi_coo, intr_coo):
    in_radius = []
    for f, frame in enumerate(resi_coo):
        in_radius.append(1 if animals_in_proximity(f, frame, intr_coo) else 0)
    return in_radius

In [12]:
# RESI MOVEMENTS
# calculates the vectors between frames, if frame or next_frame has a -1, the vectors is (0,0)
def resi_moves_vec(resi_body):
    resi_moves = []
    for i in range(len(resi_body)-1):
        resi_moves.append([0,0] if np.isin(-1, resi_body[i]) or np.isin(-1, resi_body[i+1]) else resi_body[i+1] - resi_body[i])
    resi_moves = np.array(resi_moves)
    
    # calculates vector magnitudes
    resi_moves_magn = np.array([np.linalg.norm(frame) for frame in resi_moves])
    
    return resi_moves, resi_moves_magn

In [13]:
# ANIMAL VECTORS
# calculates vectors between the resi_body and all existing intr points (from resi persepctive: intr - resi)
# if resi_body is (-1,-1) or no intr point available, vector is [0,0]
def get_vectors_between_animals(resi_body, intr_coo):
    anim_vectors = []
    for f, resi in enumerate(resi_body):
        new_frame = []
        if np.any(resi == -1) or np.all(intr_coo[f] == -1):
            new_frame.append([0,0])
        else:
            for intr in intr_coo[f]:
                if not np.any(intr == -1):
                    new_frame.append(intr - resi)
        anim_vectors.append(new_frame)
    return anim_vectors

# calculate the angle between the resi_movement_vector and the resi_intr_vector
def find_vector_angle(vector_move, vector_anim):
    dot_product = vector_move[0] * vector_anim[0] + vector_move[1] * vector_anim[1]
    mag_vector_move = math.sqrt(vector_move[0]**2 + vector_move[1]**2)
    mag_vector_anim = math.sqrt(vector_anim[0]**2 + vector_anim[1]**2)
    
    # check if the vectors are super close to zero
    if np.isclose(mag_vector_move, 0) or np.isclose(mag_vector_anim, 0):
        return -1000
    
    # check that theta is in range [-1.0, 1.0]
    theta = dot_product / (mag_vector_move * mag_vector_anim)
    theta = min(1.0, max(-1.0, theta))
    
    theta_deg = math.degrees(math.acos(theta))
    return(theta_deg)

# return list with the smallest angles between resi move and animals vectors and list with the respective animals vector
def smallest_angles_anim(vector_degrees, anim_vectors):
    smallest_angles = []
    smallest_angled_anim_vectors = []
    for f, frame in enumerate(vector_degrees):
        smallest_angles.append(min(frame))
        
        index = frame.index(min(frame))
        smallest_angled_anim_vectors.append(anim_vectors[f][index])
    smallest_angles = np.array(smallest_angles)
    return smallest_angles, smallest_angled_anim_vectors

In [14]:
# OUTPUT
# 1 = in_radius, 2 = follow, 0 = nothing
def output(in_radius, follow):
    in_radius = in_radius[:len(follow)]
    total_frames = len(in_radius)
    output = [1 if frame == 1 else 2 if follow[f] == 1 else 0 for f, frame in enumerate(in_radius)]
    output_perc = [round(output.count(value)/total_frames, 3) for value in [1, 2, 0]]
    return output_perc

In [15]:
# MAIN
def check_group(group_list):
    group = [[], [], []]
    for animal in group_list:
        csv_file = f"2023-12-13_RI1_{animal}_c.analysis.csv"
        resident, intruder = csv_into_lists(csv_file)
        resi_coo, intr_coo = clean_up_lists(resident, intruder)
        in_radius = animals_in_radius(resi_coo, intr_coo)
    
        # creates array with the resi body_center coordinates only
        resi_body = np.array([frame[2] for frame in resi_coo])
    
        resi_moves, resi_moves_magn = resi_moves_vec(resi_body)
        anim_vectors = get_vectors_between_animals(resi_body, intr_coo)
        vector_degrees = [[find_vector_angle(move, anim) for anim in anim_vectors[f]] for f, move in enumerate(resi_moves)]
        smallest_angles, smallest_angled_anim_vectors = smallest_angles_anim(vector_degrees, anim_vectors)
    
        # decide if resi is following intr (1) or not (0)
        follow = [1 if 0 <= angle <= 22.5 else 0 for angle in smallest_angles]
        output_perc = output(in_radius, follow)

        group[0].append(output_perc[0])
        group[1].append(output_perc[1])
        group[2].append(output_perc[2])
        print(f"animal {animal} done")
    return group

all_groups = []
for group in RI_1:
    all_groups.append(check_group(group))

print(all_groups)

animal 196 done
animal 198 done
animal 200 done
animal 206 done
animal 209 done
animal 211 done
animal 145 done
animal 150 done
animal 704 done
animal 706 done
animal 707 done
animal 711 done
animal 758 done
animal 767 done
animal 768 done
[[[0.259, 0.149, 0.334, 0.193, 0.182, 0.313], [0.137, 0.144, 0.132, 0.148, 0.147, 0.131], [0.605, 0.707, 0.534, 0.659, 0.671, 0.556]], [[0.599, 0.559, 0.453, 0.439, 0.259, 0.285, 0.424, 0.168, 0.357], [0.09, 0.099, 0.135, 0.116, 0.162, 0.142, 0.108, 0.174, 0.128], [0.311, 0.341, 0.412, 0.445, 0.579, 0.572, 0.468, 0.658, 0.515]]]


In [16]:
from IPython.display import display, clear_output

def draw_movie(number, resi, intr, prox_series, smallest_angles, smallest_angled_anim_vectors, resi_moves): 
    plt.clf()

    # animal points
    resi_x = [resi_coo[number][i][0] for i in range(4)]
    resi_y = [resi_coo[number][i][1] for i in range(4)]
    intr_x = [intr_coo[number][i][0] for i in range(4)]
    intr_y = [intr_coo[number][i][1] for i in range(4)]
    plt.scatter(resi_x, resi_y, label='resi', color='red')
    plt.scatter(intr_x, intr_y, label='intr', color='blue')
    
    # circle
    prox_color = "r" if in_radius[number] == 1 else "g"
    circle = plt.Circle((int(resi_x[0]), int(resi_y[0])), radius_thr, edgecolor=prox_color, facecolor='none')
    plt.gca().add_artist(circle)
    plt.gca().set_aspect('equal', adjustable='box')

    # vector arrow
    ang_color = "r" if follow[number] else "g"
    arrow = plt.arrow(*(resi_x[2], resi_y[2]), *smallest_angled_anim_vectors[number], 
                      head_width=0, head_length=0, fc=ang_color, ec=ang_color)
    arrow = plt.arrow(*(resi_x[2], resi_y[2]), *resi_moves[number]*20,
                     head_width=10, head_length=10, fc=ang_color, ec=ang_color)

    plt.xlabel('X-Pixel')
    plt.ylabel('Y-Pixel')
    plt.grid(True)
    plt.xlim(0,640)
    plt.ylim(0,350)

    plt.draw()
    plt.pause(2)

# animal points
resi_x = [resi_coo[number][i][0] for i in range(4)]
resi_y = [resi_coo[number][i][1] for i in range(4)]
intr_x = [intr_coo[number][i][0] for i in range(4)]
intr_y = [intr_coo[number][i][1] for i in range(4)]
plt.scatter(resi_x, resi_y, label='resi', color='red') 
plt.scatter(intr_x, intr_y, label='intr', color='blue')

#circle
prox_color = "r" if in_radius[number] == 1 else "g"
circle = plt.Circle((int(resi_x[0]), int(resi_y[0])), radius_thr, edgecolor=prox_color, facecolor='none')
plt.gca().add_artist(circle)
plt.gca().set_aspect('equal', adjustable='box')

#vector arrow
ang_color = "r" if follow[number] else "g" 
arrow = plt.arrow(*(resi_x[2], resi_y[2]), *smallest_angled_anim_vectors[number], head_width=0, head_length=0, fc=ang_color, ec=ang_color) 
arrow = plt.arrow(*(resi_x[2], resi_y[2]), *resi_moves[number]*20, head_width=10, head_length=10, fc=ang_color, ec=ang_color)

plt.xlabel('X-Pixel') 
plt.ylabel('Y-Pixel') 
plt.grid(True) 
plt.xlim(0,640) 
plt.ylim(0,350)

total_frames = 1000 
fig, ax = plt.subplots()


# for frame in range(total_frames): 
#     print(f"frame number = {frame*50}") 
#     draw_movie(frame*50, resi, intr, prox_series, smallest_angles, smallest_angled_anim_vectors, resi_moves) 
#     display(fig) 
#     clear_output(wait=True)

NameError: name 'resi_coo' is not defined